In [1]:
import pandas as pd
import numpy as np

In [229]:
df_original = pd.read_csv("2017_Migrant_A_desensitized2017_Migrant_A_desensitized.csv", encoding="GBK")
df_original

/opt/homebrew/anaconda3/envs/biling/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (81,82,85,86,87,88,89,90,92,93,94,97,116,160,161,162,163,164,165,166,167,169,170,171,172,173,174,175,180,181,182,183,184,186,227,228,229,230,231,232,233,234,236,237,238,239,240,241,242,243,245,246,247,248,249,250,251,252,286,288,292) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,newID,C1,C2,C3,C6,C7,C14,Q100,q101id1,q101a1,...,Q504C2,Q504D1,Q504D2,Q504E1,Q504E2,Q505,Q506,pcode,hjpro,w_l
0,105603,广东省,深圳市,光明新区,5281,居委会,5281,4,1,本人,...,本地,否,NaN,否,NaN,已经办理,1,44,36,6.719342
1,105619,广东省,深圳市,光明新区,5281,居委会,5281,3,1,本人,...,NaN,是,本地,否,NaN,已经办理,1,44,36,10.147591
2,73661,福建省,厦门市,同安区,3684,居委会,3684,4,1,本人,...,NaN,是,本地,否,NaN,已经办理,1,35,51,1.355456
3,168796,新疆生产建设兵团,第六师,军户农场,8440,居委会,8440,3,1,本人,...,本地,否,NaN,否,NaN,已经办理,2,66,65,0.157172
4,1718,北京市,北京市,朝阳区,86,居委会,86,2,1,本人,...,NaN,是,本地,否,NaN,已经办理,1,11,51,1.585694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,41331,上海市,上海市,闵行区,2067,居委会,2067,3,1,本人,...,NaN,不清楚,NaN,不清楚,NaN,没办，但听说过,1,31,33,2.686277
169985,69988,安徽省,宣城市,宁国市,3500,居委会,3500,4,1,本人,...,NaN,否,NaN,否,NaN,没办，但听说过,2,34,34,0.415267
169986,3410,北京市,北京市,海淀区,171,居委会,171,4,1,本人,...,NaN,是,本地,否,NaN,已经办理,1,11,13,1.746749
169987,55167,浙江省,杭州市,江干区,2759,居委会,2759,4,1,本人,...,NaN,是,本地,否,NaN,已经办理,2,33,33,3.795510


In [262]:
df=df_original

In [263]:
df = df.replace("是",1).replace("否",0)

In [264]:
"\',\'".join("""Q504A1
Q305A
Q305B
Q305C
Q305D
Q305E
Q305F
Q305G
Q305H""".split("\n"))

"Q504A1','Q305A','Q305B','Q305C','Q305D','Q305E','Q305F','Q305G','Q305H"

In [265]:
print("\n".join(list(df.columns)))

newID
C1
C2
C3
C6
C7
C14
Q100
q101id1
q101a1
q101b1
q101c1y
q101c1m
q101d1
q101e1
q101f1
q101g1
q101h1
q101i1
q101j1a
q101j1b
q101k1
q101l1
q101m1y
q101m1m
q101n1
q101id2
q101a2
q101b2
q101c2y
q101c2m
q101d2
q101e2
q101f2
q101g2
q101h2
q101i2
q101j2a
q101j2b
q101k2
q101l2
q101m2y
q101m2m
q101n2
q101id3
q101a3
q101b3
q101c3y
q101c3m
q101d3
q101e3
q101f3
q101g3
q101h3
q101i3
q101j3a
q101j3b
q101k3
q101l3
q101m3y
q101m3m
q101n3
q101id4
q101a4
q101b4
q101c4y
q101c4m
q101d4
q101e4
q101f4
q101g4
q101h4
q101i4
q101j4a
q101j4b
q101k4
q101l4
q101m4y
q101m4m
q101n4
q101id5
q101a5
q101b5
q101c5y
q101c5m
q101d5
q101e5
q101f5
q101g5
q101h5
q101i5
q101j5a
q101j5b
q101k5
q101l5
q101m5y
q101m5m
q101n5
q101id6
Q102
Q102A
Q102B
Q103
Q104
Q105
Q106
Q106A
Q106B
Q107
Q108A
Q108B
Q109
Q109A
Q110
Q110A
Q111
Q112
Q201
Q201A
Q202
Q203Y
Q203M
Q204
Q205
Q206
Q207
Q208
Q209
Q210Y
Q210M
Q211
Q212
Q212A
Q213Y
Q213M
Q214
Q215
Q216
Q301
Q302Y
Q302M
Q303
Q304A
Q304B
Q304D
Q305A
Q305B
Q305C
Q305D
Q305E
Q305F
Q305G
Q305

In [266]:
df["q101i1"].unique()


array([0])

In [267]:
df = df[["Q501A","Q501B","Q501C","Q501D","Q501E","Q501F",
         "Q502A","Q502B","Q502C","Q502D","Q502E",
         "Q503A","Q503D","Q503E", "Q503F", "Q503H",
         'Q504A1','Q504A2','Q504B1','Q504B2','Q504C1','Q504C2','Q504D1','Q504D2','Q504E1','Q504E2',
         'Q105','q101e1',"q101k1","q101b1",'q101d1','q101h1','Q201', 
         'Q305A','Q305B','Q305C','Q305D','Q305E','Q305F','Q305G','Q305H',
         'Q406','Q410']]

In [268]:
df["Q501_bridging"] = df.transpose()[:2].sum().transpose().apply(lambda x: 1 if x>0 else 0)
df["Q501_bounding"] = df.transpose()[2:5].sum().transpose().apply(lambda x: 1 if x>0 else 0)
df["Q501_other"] = df.transpose()[5:6].sum().transpose().apply(lambda x: 1 if x>0 else 0)

In [269]:
df["Q501"] = [" "]*len(df)
df["Q501"] [(df.Q501_bridging==1)&(df.Q501_bounding==1)] = "both"
df["Q501"] [(df.Q501==" ")&(df.Q501_bounding==1)] = "bounding"
df["Q501"] [(df.Q501==" ")&(df.Q501_bridging==1)] = "bridging"
df["Q501"] [(df.Q501==" ")&(df.Q501_other==1)] = "other"
df["Q501"] = df["Q501"].replace(" ","no group")
df=df.drop(["Q501_bridging","Q501_bounding","Q501_other"],axis=1)
df

<ipython-input-269-f3bd824a9ab4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Q501"] [(df.Q501_bridging==1)&(df.Q501_bounding==1)] = "both"
<ipython-input-269-f3bd824a9ab4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Q501"] [(df.Q501==" ")&(df.Q501_bounding==1)] = "bounding"
<ipython-input-269-f3bd824a9ab4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Q501"] [(df.Q501==" ")&(df.Q501_bridging==1)] = "bridging"
<

,Q501A,Q501B,Q501C,Q501D,Q501E,Q501F,Q502A,Q502B,Q502C,Q502D,...,Q305B,Q305C,Q305D,Q305E,Q305F,Q305G,Q305H,Q406,Q410,Q501
0,0,0,0,0,0,0,偶尔,没有,没有,偶尔,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,均未患有,0,no group
1,0,0,0,0,0,0,没有,没有,没有,没有,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,均未患有,0,no group
2,0,0,1,0,0,1,没有,没有,没有,偶尔,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,均未患有,0,bounding
3,0,0,1,0,0,0,没有,没有,没有,偶尔,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,均未患有,0,bounding
4,0,0,0,0,0,0,没有,没有,没有,没有,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,均未患有,0,no group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,0,0,0,0,0,0,没有,没有,没有,没有,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,均未患有,0,no group
169985,0,0,0,0,0,0,没有,没有,没有,偶尔,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,患有高血压,0,no group
169986,0,0,1,1,0,0,偶尔,没有,没有,没有,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,均未患有,是，最近一次发生在两周前,bounding
169987,0,0,0,1,0,0,经常,偶尔,没有,经常,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,均未患有,是，最近一次发生在两周前,bounding


In [270]:
df["Q502"] = df["Q502A"]+df["Q502B"]+df["Q502C"]+df["Q502D"]+df["Q502E"]
def map502(x):
    if "经常" in x:
        return 4
    elif "有时" in x:
        return 3
    elif "偶尔" in x:
        return 2
    elif "没有" in x:
        return 1
    else:
        return np.nan
df["Q502"] = df["Q502"].apply(lambda x: map502(x))

In [271]:
df = df.drop(["Q501A","Q501B","Q501C","Q501D","Q501E","Q501F",
         "Q502A","Q502B","Q502C","Q502D","Q502E"], axis=1)

In [299]:
import numpy as np

d = {"经常": 4,
     "有时": 3,
     "偶尔": 2,
     "没有":1,
     '完全同意':2,
     '基本同意':1,
     '不同意':-1,
     '完全不同意':-2,
     "不清楚": np.nan,
     "女":"female",
     "男":"male",
     '未上过小学':"no education",
     '小学':"primary school",
     '初中':"midschool", 
     '高中/中专':"highschool",
     '大学专科':"junior college",
     '大学本科':"college",
     '研究生':"grad",
     '女':1,
     '男':2,
     '汉':1,
     '回':2,
     '蒙':3, 
     '彝':4, 
     '土家':5, 
     '苗':6, 
     '白':7, 
     '藏':8, 
     '侗':9,
     '其他':"other", 
     '满':10, 
     '朝鲜':11,
     '黎':12, 
     '维吾尔':13, 
     '哈萨克':14, 
     '哈尼':15, 
     '傣':16, 
     '布依':17, 
     '瑶':18,
     '未婚':"unmarried",
     '同居':"living together",
     '初婚':"just married",
     '再婚':"remarried", 
     '离婚':"divorced",
     '丧偶':"widowed",
     '均未患有':0, 
     '未就诊':np.nan, 
     '患有糖尿病':"db", 
     '患有高血压和糖尿病':"db+hpt", 
     '患有高血压':"hpt",
     '健康': 4, 
     '基本健康': 3, 
     '不健康，但生活能自理': 2, 
     '生活不能自理': 1,
     '是，最近一次发生在两周前': "within 2 weeks", 
     '是，最近一次发生在两周内': "before 2 weeks",
     '跨省':"interstate",
     '省内跨市':"intercity", 
     '市内跨县':"intercounty",
     '有固定雇主的雇员':"stable job",
     '雇主':"stable job", 
     '自营劳动者':"stable job", 
     '其他':"other",
     '无固定雇主的雇员（零工、散工等）':"unstable job",
     '愿意':1, 
     '不愿意':-1, 
     '没想好':0,
     '是':1,
     '没想好':0, 
     '否':-1
    }

In [273]:
def replace(x):
    for k,v in d.items():
        if x == k:
            return v
    return x
df=df.applymap(lambda x: replace(x))
df

,Q503A,Q503D,Q503E,Q503F,Q503H,Q504A1,Q504A2,Q504B1,Q504B2,Q504C1,...,Q305C,Q305D,Q305E,Q305F,Q305G,Q305H,Q406,Q410,Q501,Q502
0,1,-1,1,-1,-1,0.0,NaN,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,no group,2
1,2,1,1,-1,2,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0,0,no group,1
2,2,1,1,1,2,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,bounding,2
3,1,1,-1,-1,2,0.0,NaN,0.0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,bounding,2
4,1,1,-2,1,1,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,no group,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,2,2,-2,-2,2,1.0,户籍地,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,no group,1
169985,2,2,-2,1,1,1.0,户籍地,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,hpt,0,no group,2
169986,2,2,-1,1,2,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,within 2 weeks,bounding,2
169987,2,2,-1,1,2,0.0,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,within 2 weeks,bounding,4


In [274]:
df["Q504"] = df['Q504A1'].astype(float)+df['Q504B1'].astype(float)+df['Q504C1'].astype(float)+df['Q504D1'].astype(float)+df['Q504E1'].astype(float)
df["Q504"] = df["Q504"].apply(lambda x: 1 if x>0 else 0).astype(int)
df = df.drop([ 'Q504A1','Q504A2','Q504B1','Q504B2','Q504C1','Q504C2','Q504D1','Q504D2','Q504E1','Q504E2'],axis=1)
df

,Q503A,Q503D,Q503E,Q503F,Q503H,Q105,q101e1,q101k1,q101b1,q101d1,...,Q305D,Q305E,Q305F,Q305G,Q305H,Q406,Q410,Q501,Q502,Q504
0,1,-1,1,-1,-1,NaN,highschool,本地,1,1,...,NaN,NaN,NaN,NaN,NaN,0,0,no group,2,1
1,2,1,1,-1,2,NaN,junior college,本地,1,1,...,0.0,0.0,1.0,0.0,0.0,0,0,no group,1,0
2,2,1,1,1,2,NaN,highschool,本地,1,1,...,NaN,NaN,NaN,NaN,NaN,0,0,bounding,2,1
3,1,1,-1,-1,2,NaN,midschool,本地,2,1,...,0.0,0.0,0.0,0.0,0.0,0,0,bounding,2,1
4,1,1,-2,1,1,NaN,junior college,本地,1,1,...,NaN,NaN,NaN,NaN,NaN,0,0,no group,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,2,2,-2,-2,2,100000.0,primary school,本地,1,1,...,0.0,0.0,0.0,0.0,0.0,0,0,no group,1,0
169985,2,2,-2,1,1,120000.0,midschool,本地,2,1,...,NaN,NaN,NaN,NaN,NaN,hpt,0,no group,2,1
169986,2,2,-1,1,2,180000.0,college,本地,1,10,...,NaN,NaN,NaN,NaN,NaN,0,within 2 weeks,bounding,2,1
169987,2,2,-1,1,2,200000.0,midschool,本地,1,1,...,NaN,NaN,NaN,NaN,NaN,0,within 2 weeks,bounding,4,1


In [275]:
df["q101k1"].unique()

array(['本地'], dtype=object)

In [276]:
df=df.drop("q101k1",axis=1)

In [277]:
df["q101b1"] = df_original["q101b1"]
df["q101b1"].unique()

array(['女', '男'], dtype=object)

In [278]:
def sex(x):
    if x=="女":
        return "female" 
    elif x=="男":
        return "male"
    else:
        return np.nan
df["q101b1"] = df["q101b1"].map(lambda x: sex(x))
df["q101b1"]

0         female
1         female
2         female
3           male
4         female
           ...  
169984    female
169985      male
169986    female
169987    female
169988    female
Name: q101b1, Length: 169989, dtype: object

In [279]:
df["Q305_type1"] = df[['Q305A']].replace(np.nan,0)
df["Q305_type1"].unique()



array([1, 0])

In [280]:
df["q305_2"]=df[['Q305B','Q305C','Q305D','Q305E','Q305F']].replace(np.nan,0).sum(axis=1)
df["Q305_type2"] = df["q305_2"].apply(lambda x: 1 if x>0 else 0)
df["Q305_type2"].unique()


array([0, 1])

In [281]:
df["q305_2"]=df[['Q305G','Q305H']].replace(np.nan,0).sum(axis=1)
df["Q305_type3"] = df["q305_2"].apply(lambda x: 1 if x>0 else 0)
df["Q305_type3"].unique()
df = df.drop("q305_2",axis=1)

In [282]:
df=df.drop([ 'Q305A','Q305B','Q305C','Q305D','Q305E','Q305F','Q305G','Q305H'],axis=1)

In [283]:
df

,Q503A,Q503D,Q503E,Q503F,Q503H,Q105,q101e1,q101b1,q101d1,q101h1,Q201,Q406,Q410,Q501,Q502,Q504,Q305_type1,Q305_type2,Q305_type3
0,1,-1,1,-1,-1,NaN,highschool,female,1,just married,1,0,0,no group,2,1,1,0,0
1,2,1,1,-1,2,NaN,junior college,female,1,remarried,0,0,0,no group,1,0,0,1,0
2,2,1,1,1,2,NaN,highschool,female,1,just married,1,0,0,bounding,2,1,1,0,0
3,1,1,-1,-1,2,NaN,midschool,male,1,just married,1,0,0,bounding,2,1,0,1,0
4,1,1,-2,1,1,NaN,junior college,female,1,just married,1,0,0,no group,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,2,2,-2,-2,2,100000.0,primary school,female,1,just married,0,0,0,no group,1,0,0,1,0
169985,2,2,-2,1,1,120000.0,midschool,male,1,just married,1,hpt,0,no group,2,1,1,0,0
169986,2,2,-1,1,2,180000.0,college,female,10,just married,1,0,within 2 weeks,bounding,2,1,1,0,0
169987,2,2,-1,1,2,200000.0,midschool,female,1,just married,1,0,within 2 weeks,bounding,4,1,1,0,0


In [284]:
df = df.reset_index()
df.insert(0,"participant",df["index"].astype(str))
df=df.drop("index",axis=1)
df

,participant,Q503A,Q503D,Q503E,Q503F,Q503H,Q105,q101e1,q101b1,q101d1,q101h1,Q201,Q406,Q410,Q501,Q502,Q504,Q305_type1,Q305_type2,Q305_type3
0,0,1,-1,1,-1,-1,NaN,highschool,female,1,just married,1,0,0,no group,2,1,1,0,0
1,1,2,1,1,-1,2,NaN,junior college,female,1,remarried,0,0,0,no group,1,0,0,1,0
2,2,2,1,1,1,2,NaN,highschool,female,1,just married,1,0,0,bounding,2,1,1,0,0
3,3,1,1,-1,-1,2,NaN,midschool,male,1,just married,1,0,0,bounding,2,1,0,1,0
4,4,1,1,-2,1,1,NaN,junior college,female,1,just married,1,0,0,no group,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984,2,2,-2,-2,2,100000.0,primary school,female,1,just married,0,0,0,no group,1,0,0,1,0
169985,169985,2,2,-2,1,1,120000.0,midschool,male,1,just married,1,hpt,0,no group,2,1,1,0,0
169986,169986,2,2,-1,1,2,180000.0,college,female,10,just married,1,0,within 2 weeks,bounding,2,1,1,0,0
169987,169987,2,2,-1,1,2,200000.0,midschool,female,1,just married,1,0,within 2 weeks,bounding,4,1,1,0,0


In [285]:
df["Q401"] = df_original["Q401"]
d = dict(zip(['健康', '基本健康', '不健康，但生活能自理', '生活不能自理'],[4,3,2,1]))
for k,v in d.items():
    df["Q401"] = df["Q401"].replace(k,v)
df.groupby("Q401").agg("count")

,participant,Q503A,Q503D,Q503E,Q503F,Q503H,Q105,q101e1,q101b1,q101d1,q101h1,Q201,Q406,Q410,Q501,Q502,Q504,Q305_type1,Q305_type2,Q305_type3
Q401,,,,,,,,,,,,,,,,,,,,
1,167,167,167,167,167,167,167,167,167,167,167,167,160,167,167,167,167,167,167,167
2,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4438,4239,4438,4438,4438,4438,4438,4438,4438
3,25694,25694,25694,25694,25694,25694,25692,25694,25694,25694,25694,25694,24637,25694,25694,25694,25694,25694,25694,25694
4,139690,139690,139690,139690,139690,139690,139685,139690,139690,139690,139690,139690,137116,139690,139690,139690,139690,139690,139690,139690


In [286]:
df.columns = columns[:21]

In [287]:
df.to_csv("preliminary_data.csv",index=False)


In [288]:
df


,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,worked_before5.1,diabete_hypertension,recent_disease,group,participated_in_group_activity,insuranced,migration.self,migration.relatives,migration.fellows,health
0,0,1,-1,1,-1,-1,NaN,highschool,female,1,...,1,0,0,no group,2,1,1,0,0,4
1,1,2,1,1,-1,2,NaN,junior college,female,1,...,0,0,0,no group,1,0,0,1,0,3
2,2,2,1,1,1,2,NaN,highschool,female,1,...,1,0,0,bounding,2,1,1,0,0,4
3,3,1,1,-1,-1,2,NaN,midschool,male,1,...,1,0,0,bounding,2,1,0,1,0,4
4,4,1,1,-2,1,1,NaN,junior college,female,1,...,1,0,0,no group,1,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984,2,2,-2,-2,2,100000.0,primary school,female,1,...,0,0,0,no group,1,0,0,1,0,4
169985,169985,2,2,-2,1,1,120000.0,midschool,male,1,...,1,hpt,0,no group,2,1,1,0,0,4
169986,169986,2,2,-1,1,2,180000.0,college,female,10,...,1,0,within 2 weeks,bounding,2,1,1,0,0,3
169987,169987,2,2,-1,1,2,200000.0,midschool,female,1,...,1,0,within 2 weeks,bounding,4,1,1,0,0,4


## processed

In [257]:
import pandas as pd
df = pd.read_csv("prcessed_data.csv")
df

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,group,participated_in_group_activity,insuranced,migration.self,migration.relatives,migration.fellows,health,migration.type,income.normalized,income.thousand
0,0.0,1,-1,1,-1,-1,NaN,3,1,1,...,no_group,2,1,1,0,0,4,self,NaN,NaN
1,1.0,2,1,1,-1,2,NaN,4,1,1,...,no_group,1,0,0,1,0,3,relatives,NaN,NaN
2,2.0,2,1,1,1,2,NaN,3,1,1,...,bounding,2,1,1,0,0,4,self,NaN,NaN
3,3.0,1,1,-1,-1,2,NaN,2,2,1,...,bounding,2,1,0,1,0,4,relatives,NaN,NaN
4,4.0,1,1,-2,1,1,NaN,4,1,1,...,no_group,1,1,1,0,0,4,self,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984.0,2,2,-2,-2,2,100000.0,1,1,1,...,no_group,1,0,0,1,0,4,relatives,16.124701,100.0
169985,169985.0,2,2,-2,1,1,120000.0,2,2,1,...,no_group,2,1,1,0,0,4,self,19.597445,120.0
169986,169986.0,2,2,-1,1,2,180000.0,5,1,10,...,bounding,2,1,1,0,0,3,self,30.015678,180.0
169987,169987.0,2,2,-1,1,2,200000.0,2,1,1,...,bounding,4,1,1,0,0,4,self,33.488422,200.0


In [258]:
columns = df.columns
len(columns)

24

In [259]:
df[df.group=="no_group"]

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,group,participated_in_group_activity,insuranced,migration.self,migration.relatives,migration.fellows,health,migration.type,income.normalized,income.thousand
0,0.0,1,-1,1,-1,-1,NaN,3,1,1,...,no_group,2,1,1,0,0,4,self,NaN,NaN
1,1.0,2,1,1,-1,2,NaN,4,1,1,...,no_group,1,0,0,1,0,3,relatives,NaN,NaN
4,4.0,1,1,-2,1,1,NaN,4,1,1,...,no_group,1,1,1,0,0,4,self,NaN,NaN
11,11.0,2,1,-1,-1,2,-20000.0,3,1,1,...,no_group,2,1,0,1,1,3,relatives,-4.711765,-20.0
14,14.0,1,1,1,1,-2,-12500.0,2,2,1,...,no_group,2,1,0,1,1,4,relatives,-3.409485,-12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169973,169973.0,1,-1,1,-2,1,90000.0,2,2,1,...,no_group,1,1,1,0,0,4,self,14.388329,90.0
169974,169974.0,1,1,2,-1,-1,90000.0,3,2,1,...,no_group,1,0,1,0,0,3,self,14.388329,90.0
169977,169977.0,2,-1,-1,-1,1,99000.0,5,2,1,...,no_group,2,1,1,0,0,4,self,15.951064,99.0
169984,169984.0,2,2,-2,-2,2,100000.0,1,1,1,...,no_group,1,0,0,1,0,4,relatives,16.124701,100.0


In [260]:
df[df.group=="both"]

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,group,participated_in_group_activity,insuranced,migration.self,migration.relatives,migration.fellows,health,migration.type,income.normalized,income.thousand
44,44.0,2,1,-1,1,1,-2750.0,2,2,1,...,both,2,1,1,0,0,4,self,-1.716523,-2.75
130,130.0,2,2,-2,-2,2,0.0,1,2,2,...,both,2,1,0,1,0,2,relatives,-1.239020,0.00
169,169.0,2,2,-1,-1,1,0.0,3,1,1,...,both,2,1,1,0,0,4,self,-1.239020,0.00
177,177.0,2,2,-1,-1,1,0.0,5,2,1,...,both,2,1,1,0,0,4,self,-1.239020,0.00
196,196.0,1,1,-2,-1,1,0.0,5,1,1,...,both,2,1,0,1,0,4,relatives,-1.239020,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169956,169956.0,2,2,-2,-1,-1,90000.0,5,2,1,...,both,3,1,1,0,0,4,self,14.388329,90.00
169960,169960.0,1,1,-1,-1,1,90000.0,6,2,1,...,both,4,0,1,0,0,4,self,14.388329,90.00
169962,169962.0,1,1,-1,-1,2,90000.0,3,1,1,...,both,2,1,1,0,0,4,self,14.388329,90.00
169978,169978.0,2,2,-1,-1,-1,100000.0,6,2,1,...,both,2,1,1,0,0,4,self,16.124701,100.00


In [261]:
print(" + ".join(df.columns))

participant + like.current.city + natives.like.me + natives.lookdown.me + native.customs.better + i.am.native + income + education + gender + ethnicity + marriage + worked_before5.1 + diabete_hypertension + recent_disease + group + participated_in_group_activity + insuranced + migration.self + migration.relatives + migration.fellows + health + migration.type + income.normalized + income.thousand


In [10]:
for k,v in d.items():
    df=df.replace(k,v)
df

,participant,Q503A,Q503D,Q503E,Q503F,Q503H,Q105,q101e1,q101b1,q101d1,...,Q201,Q406,Q410,Q501,Q502,Q504,Q305_type1,Q305_type2,Q305_type3,Q401
0,0,1,-1,1,-1,-1,NaN,3,1,1,...,1,0.0,0,,2,1,1,0,0,1
1,1,2,1,1,-1,2,NaN,4,1,1,...,0,0.0,0,,1,0,0,1,0,2
2,2,2,1,1,1,2,NaN,3,1,1,...,1,0.0,0,bounding,2,1,1,0,0,1
3,3,1,1,-1,-1,2,NaN,2,2,1,...,1,0.0,0,bounding,2,1,0,1,0,1
4,4,1,1,-2,1,1,NaN,4,1,1,...,1,0.0,0,,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984,2,2,-2,-2,2,100000.0,1,1,1,...,0,0.0,0,,1,0,0,1,0,1
169985,169985,2,2,-2,1,1,120000.0,2,2,1,...,1,2.0,0,,2,1,1,0,0,1
169986,169986,2,2,-1,1,2,180000.0,5,1,10,...,1,0.0,1,bounding,2,1,1,0,0,2
169987,169987,2,2,-1,1,2,200000.0,2,1,1,...,1,0.0,1,bounding,4,1,1,0,0,1


In [86]:
instruction = pd.DataFrame.from_dict(d,orient='index')
instruction.columns = ["value"]
instruction

,value
经常,4
有时,3
偶尔,2
没有,1
完全同意,2
基本同意,1
不同意,-1
完全不同意,-2
不清楚,NaN
未上过小学,0


In [87]:
instruction.to_csv("instruction.csv")


In [205]:
print("\n".join(df_original.columns))

newID
C1
C2
C3
C6
C7
C14
Q100
q101id1
q101a1
q101b1
q101c1y
q101c1m
q101d1
q101e1
q101f1
q101g1
q101h1
q101i1
q101j1a
q101j1b
q101k1
q101l1
q101m1y
q101m1m
q101n1
q101id2
q101a2
q101b2
q101c2y
q101c2m
q101d2
q101e2
q101f2
q101g2
q101h2
q101i2
q101j2a
q101j2b
q101k2
q101l2
q101m2y
q101m2m
q101n2
q101id3
q101a3
q101b3
q101c3y
q101c3m
q101d3
q101e3
q101f3
q101g3
q101h3
q101i3
q101j3a
q101j3b
q101k3
q101l3
q101m3y
q101m3m
q101n3
q101id4
q101a4
q101b4
q101c4y
q101c4m
q101d4
q101e4
q101f4
q101g4
q101h4
q101i4
q101j4a
q101j4b
q101k4
q101l4
q101m4y
q101m4m
q101n4
q101id5
q101a5
q101b5
q101c5y
q101c5m
q101d5
q101e5
q101f5
q101g5
q101h5
q101i5
q101j5a
q101j5b
q101k5
q101l5
q101m5y
q101m5m
q101n5
q101id6
Q102
Q102A
Q102B
Q103
Q104
Q105
Q106
Q106A
Q106B
Q107
Q108A
Q108B
Q109
Q109A
Q110
Q110A
Q111
Q112
Q201
Q201A
Q202
Q203Y
Q203M
Q204
Q205
Q206
Q207
Q208
Q209
Q210Y
Q210M
Q211
Q212
Q212A
Q213Y
Q213M
Q214
Q215
Q216
Q301
Q302Y
Q302M
Q303
Q304A
Q304B
Q304D
Q305A
Q305B
Q305C
Q305D
Q305E
Q305F
Q305G
Q305

In [302]:
## more variables to study

comp = df_original[["q101l1","Q104","Q208","Q313","Q314"]]
comp


,q101l1,Q104,Q208,Q313,Q314
0,跨省,10000.0,有固定雇主的雇员,愿意,是
1,跨省,60000.0,NaN,愿意,是
2,跨省,40000.0,雇主,愿意,是
3,省内跨市,5000.0,自营劳动者,不愿意,是
4,跨省,9000.0,有固定雇主的雇员,愿意,是
...,...,...,...,...,...
169984,跨省,10000.0,NaN,不愿意,是
169985,市内跨县,15000.0,雇主,不愿意,是
169986,跨省,120000.0,有固定雇主的雇员,愿意,是
169987,省内跨市,100000.0,雇主,愿意,是


In [303]:
comp = comp.applymap(lambda x:replace(x))

In [304]:
comp

,q101l1,Q104,Q208,Q313,Q314
0,interstate,10000.0,stable job,1,1
1,interstate,60000.0,NaN,1,1
2,interstate,40000.0,stable job,1,1
3,intercity,5000.0,stable job,-1,1
4,interstate,9000.0,stable job,1,1
...,...,...,...,...,...
169984,interstate,10000.0,NaN,-1,1
169985,intercounty,15000.0,stable job,-1,1
169986,interstate,120000.0,stable job,1,1
169987,intercity,100000.0,stable job,1,1


In [305]:
comp.columns = ["migration_scale","expence","job","willing.to.movein","willing.to.stay"]

In [306]:
df = pd.concat([df.T,comp.T]).T

In [307]:
df

,participant,like.current.city,natives.like.me,natives.lookdown.me,native.customs.better,i.am.native,income,education,gender,ethnicity,...,insuranced,migration.self,migration.relatives,migration.fellows,health,migration_scale,expence,job,willing.to.movein,willing.to.stay
0,0,1,-1,1,-1,-1,NaN,highschool,female,1,...,1,1,0,0,4,interstate,10000.0,stable job,1,1
1,1,2,1,1,-1,2,NaN,junior college,female,1,...,0,0,1,0,3,interstate,60000.0,NaN,1,1
2,2,2,1,1,1,2,NaN,highschool,female,1,...,1,1,0,0,4,interstate,40000.0,stable job,1,1
3,3,1,1,-1,-1,2,NaN,midschool,male,1,...,1,0,1,0,4,intercity,5000.0,stable job,-1,1
4,4,1,1,-2,1,1,NaN,junior college,female,1,...,1,1,0,0,4,interstate,9000.0,stable job,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169984,169984,2,2,-2,-2,2,100000.0,primary school,female,1,...,0,0,1,0,4,interstate,10000.0,NaN,-1,1
169985,169985,2,2,-2,1,1,120000.0,midschool,male,1,...,1,1,0,0,4,intercounty,15000.0,stable job,-1,1
169986,169986,2,2,-1,1,2,180000.0,college,female,10,...,1,1,0,0,3,interstate,120000.0,stable job,1,1
169987,169987,2,2,-1,1,2,200000.0,midschool,female,1,...,1,1,0,0,4,intercity,100000.0,stable job,1,1


In [308]:
df.to_csv("preliminary_data.csv",index=False)
